<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Simple Decision Prompts

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal of prompt engineering is to "tune" the prompt in a way that maximizes the quality and clarity of the AI's response, often using specific wording, context, or examples within the prompt itself.

Decision Intelligence and Prompt Engineering techniques can be combined to created "Generative AI Decision Intellignce"! This can leverage GenAI models to reason over complex tasks, gather intelligence for decisions, recommend decisions, communicate decisions etc. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.54"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.54.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [ ]:
// A Decision Intelligence prompt to help with describing decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
var response = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

Certainly! Here are **five decision-making frameworks** that enhance the quality of decisions, each with a brief description of how they support better analysis and reasoning:

---

### 1. **The SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
**Description:**  
SWOT is a strategic tool for evaluating internal and external factors affecting a decision. By systematically listing Strengths, Weaknesses, Opportunities, and Threats, it provides a holistic view of the situation.  
**How it helps:**  
- Encourages comprehensive environmental scanning  
- Facilitates balanced analysis of pros and cons  
- Helps prioritize actions based on a clear assessment  

---

### 2. **The Decision Matrix (Weighted Scoring Model)**
**Description:**  
Also known as the Multi-Criteria Decision Analysis (MCDA), this method involves listing options and criteria, assigning weights to each criterion, and scoring each option.  
**How it helps:**  
- Quantifies qualitative factors for objective comparisons  
- Clarifies trade-offs by revealing which criteria matter most  
- Streamlines complex choices with multiple variables  

---

### 3. **Cost-Benefit Analysis (CBA)**
**Description:**  
CBA involves listing all costs and benefits of each alternative, often in monetary terms, to identify the most advantageous option.  
**How it helps:**  
- Reveals the financial implications and expected value  
- Assists in prioritizing options with the greatest net benefit  
- Makes reasoning transparent and justifiable  

---

### 4. **OODA Loop (Observe, Orient, Decide, Act)**
**Description:**  
Originally developed for military strategy, the OODA Loop promotes fast, adaptive decision-making by cycling through observing, orienting, deciding, and acting.  
**How it helps:**  
- Encourages continual reassessment as new information arises  
- Helps adapt decisions in dynamic, fast-changing environments  
- Structures thinking to avoid paralysis by analysis  

---

### 5. **Kepner-Tregoe Decision Analysis**
**Description:**  
This systematic process involves defining the problem, evaluating alternatives using specified criteria, assessing potential risks, and making a rational choice.  
**How it helps:**  
- Distinguishes between must-have and nice-to-have criteria  
- Anticipates and mitigates potential risks  
- Promotes clear documentation and accountability in decisions  

---

Each framework supports better decisions by introducing structure, objectivity, and clarity to the analysis, tailored to various contexts such as business strategy, project management, and rapid-response scenarios.

### Step 3 - Execute a Decision Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> 📝 Note: An average human can read between 25-40 Tokens / second. Therefore, wile streaming certainly helps with providing AI output to the user, it begins to lose its effectivness at large token velocity.  

In [5]:
// Decision Prompt using Streaming output chunks 
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Certainly! Here are **five decision-making frameworks**, along with a brief description of how each supports better analysis and reasoning:

---

### 1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
**Description:**  
SWOT provides a structured way to evaluate the internal and external factors affecting a decision.  
**How it helps:**  
By categorizing aspects into strengths, weaknesses, opportunities, and threats, it ensures consideration of all relevant factors. This comprehensive view supports objective assessment and highlights potential risks and advantages in strategic planning or business decisions.

---

### 2. **Decision Matrix (Weighted Scoring Model)**
**Description:**  
A Decision Matrix helps compare multiple options across defined criteria by assigning weights and scores.  
**How it helps:**  
It quantifies subjective judgments and enables transparent comparison. This is especially useful for prioritizing projects, selecting vendors, or choosing among 

### Step 4 - Execute a Decision Prompt with Improved Output Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can ouput Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting ouptut. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [7]:
// A new decision prompt to help with describing decision-making frameworks using Markdown format
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each decision-making framework supports better analysis and reasoning in various scenarios.

Format the response using only a Markdown table. Do not enclose the table in triple backticks.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat);
var response = simplePromptResponse.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework                | Description & How It Supports Better Decisions                                                                                                                                                   |
|--------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis            | Evaluates Strengths, Weaknesses, Opportunities, and Threats to provide a structured overview, enabling balanced consideration of internal and external factors.                                  |
| Cost-Benefit Analysis    | Compares the costs and benefits of alternatives quantitatively, helping to rationalize choices based on maximizing net value or minimizing loss.                                                  |
| Decision Matrix          | Uses weighted criteria to score and compare options, providing clear, objective comparisons and reducing bias in selection.                                                                       |
| Six Thinking Hats        | Encourages looking at a problem from multiple perspectives—emotional, logical, creative, etc.—to ensure diverse viewpoints and comprehensive reasoning are applied.                              |
| OODA Loop                | The cycle of Observe, Orient, Decide, and Act fosters rapid adaptation and continual reassessment, enhancing decision agility in dynamic or uncertain environments.                              |

### Step 5 - Execute a Decision Prompt with a Custom Prompt Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

> 📝 Note: The supported settings are dependendent on the API plus the specific model version. For example, a model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available.  

In [8]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Crete a new KernelArguments object with the OpenAI prompt execution settings
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseWithOpenAI = await semanticKernel.InvokePromptAsync(simpleDecisionPromptWithMarkdownFormat, kernelArguments);
var response = promptResponseWithOpenAI.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| Framework                | Description                                                                                                 | How It Enhances Decision Quality                                                                                 |
|--------------------------|-------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis            | Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.               | Encourages comprehensive internal and external analysis, clarifying risks and advantages for informed choices.    |
| Cost-Benefit Analysis    | Compares the expected costs and benefits of different options.                                               | Quantifies trade-offs, supporting rational, objective decision-making based on measurable outcomes.               |
| Decision Matrix          | Rates and weights multiple criteria to compare alternatives systematically.                                  | Reduces bias by providing a structured, transparent process for evaluating complex, multi-factor decisions.       |
| Six Thinking Hats        | Uses six distinct perspectives (hats) to explore a problem from emotional, logical, creative, and critical angles. | Promotes balanced, holistic thinking by ensuring diverse viewpoints are considered before making a decision.      |
| OODA Loop                | Stands for Observe, Orient, Decide, Act; a cyclical process for rapid, adaptive decision-making.             | Enhances agility and responsiveness, allowing for continuous reassessment and adjustment in dynamic environments. |

### Step 6 - Execute a Decision Prompt with a System Prompt (Custom AI Persona)

When building Decison Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific (i.e. decision intelligence)
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something else
* Provide context (additional information) to the AI
* Using Roles in Chat Completion API prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a unique persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [9]:
// Create a System Prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Console.WriteLine("PROMPT TO SEND TO THE CHAT COMPLETION SERVICE...");
Console.WriteLine(string.Empty);
Console.WriteLine(simpleDecisionPromptTemplate);
Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("RESPONSE FROM THE CHAT COMPLETION SERVICE...");
Console.WriteLine(string.Empty);


var promptResponseWithTemplate = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt, kernelArguments);
var response = promptResponseWithTemplate.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

PROMPT TO SEND TO THE CHAT COMPLETION SERVICE...

System Prompt: 
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.

Request from the user: 
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply to various scenarios.

----------------------------------------

RESPONSE FROM THE CHAT COMPLETION SERVICE...



Certainly! Here are the top 5 decision frameworks that are easy to understand, simple to apply, and versatile for daily situations:

---

### 1. **Pros and Cons List**
**How it works:**  
List the positive (pros) and negative (cons) aspects of each option. Compare the lists to see which option has more advantages or fewer disadvantages.

**When to use:**  
- Everyday choices (e.g., what to eat, which route to take)
- Weighing simple alternatives

**Why it works:**  
It clarifies your thoughts and helps visualize trade-offs.

---

### 2. **Eisenhower Matrix (Urgent-Important Matrix)**
**How it works:**  
Categorize tasks or decisions into four quadrants:
- Urgent & Important
- Important, Not Urgent
- Urgent, Not Important
- Not Urgent, Not Important

**When to use:**  
- Prioritizing tasks
- Deciding what to do now, schedule, delegate, or drop

**Why it works:**  
It helps focus on what truly matters and reduces overwhelm.

---

### 3. **The 2-Minute Rule**
**How it works:**  
If a decision or task can be completed in two minutes or less, do it immediately. If not, decide whether to schedule, delegate, or discard it.

**When to use:**  
- Quick decisions
- Managing small tasks and avoiding procrastination

**Why it works:**  
It keeps you moving and prevents small decisions from piling up.

---

### 4. **The 10/10/10 Rule**
**How it works:**  
Ask yourself:  
- How will I feel about this decision in 10 minutes?  
- In 10 months?  
- In 10 years?

**When to use:**  
- Emotional or impulsive decisions
- Choices with potential long-term impact

**Why it works:**  
It encourages long-term thinking and perspective.

---

### 5. **OODA Loop (Observe, Orient, Decide, Act)**
**How it works:**  
- **Observe:** Gather information.
- **Orient:** Analyze and interpret the information.
- **Decide:** Choose a course of action.
- **Act:** Implement the decision.

**When to use:**  
- Situations requiring quick, iterative decisions
- Problem-solving in dynamic environments

**Why it works:**  
It’s cyclical, so you can adjust as you go, making it practical for daily life.

---

**Summary Table:**

| Framework          | Best For                       | Key Benefit                |
|--------------------|-------------------------------|----------------------------|
| Pros & Cons List   | Simple choices                 | Clarifies trade-offs       |
| Eisenhower Matrix  | Task prioritization            | Focuses on importance      |
| 2-Minute Rule      | Quick, small decisions         | Reduces procrastination    |
| 10/10/10 Rule      | Emotional/long-term decisions  | Adds perspective           |
| OODA Loop          | Dynamic, fast decisions        | Encourages adaptability    |

---

**Tip:**  
You can mix and match these frameworks depending on the situation. For example, use a Pros and Cons list for a big decision, then apply the 10/10/10 Rule to check your perspective.

Would you like examples of how to use any of these frameworks in a specific daily scenario?

### Step 7 - Execute a Decision Scenario with a System Prompt (Custom AI Persona)

In this step a decision scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full Decision Intelligence framework will not be used, rather a simple request for Artificial Intelligene to recommend a path forward (recommedation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are Alex's main decision factors: financial, career uncertainty, academic consistency and future impact. In addition, you have all the decision factor detailed data available to pass to the GenAI model prompt. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence be that impartial judge to help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [10]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed Decision Scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommedation in a Markdown table format. 

Financial Considerations:
Alex's four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor's degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to build early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. 

Future Impact: 
Alex is unsure of the short-term future impact of her decision that might be hard to remediate. 
Alex wants a solid professional network and relevant experience when she graduates. 
Alex is not overly concerned about the social aspect of college, 
but feels she can build a quality network in a four-year university setting sooner. 
She is also concerned about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. 
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: 
{systemPromptForDecisions}

Request from the user: 
{scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 1000, // Increase the max tokens to allow for a more detailed response
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

var promptResponseScenario = await semanticKernel.InvokePromptAsync(scenarioDecisionPromptTemplate, kernelArguments);
var response = promptResponseScenario.GetValue<string>();

// Display the response from the Semantic Kernel as Markdown
response.DisplayAs("text/markdown");

| **Decision Factor**                | **Four-Year University (Direct Entry)**                                                                 | **Community College (Associate First, Then Transfer)**                                         | **Analysis & Recommendation**                                                                                                                                                                                                                                                                                                                                                     |
|------------------------------------|--------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Financial Considerations**       | High cost ($50k-$60k/year); family can afford with some loans; medium concern.                         | Low cost ($3k-$5k/year); significant savings; may require loans for later transfer.            | Community college offers substantial savings, reducing future debt and financial pressure. While the family can afford the university, minimizing debt is prudent, especially given Alex’s uncertainty about her major and career path.                                                                                                                                            |
| **Career & Major Uncertainty**     | Early commitment to a major; switching majors could increase time/cost; less flexibility to explore.   | More flexibility to explore majors at lower cost; easier to change direction early on.         | Community college provides a low-risk environment for academic exploration, which is ideal given Alex’s uncertainty. This reduces the risk of costly mistakes if she changes majors.                                                                                                                |
| **Academic Consistency & Networking** | Immediate access to university resources, professors, and networking; early campus integration.        | Delayed access to university-level networking; possible adjustment challenges when transferring. | While direct entry offers earlier networking, community college students can still build strong networks after transferring. Many universities have robust transfer programs to integrate new students. The initial delay is offset by the financial and academic flexibility benefits.                                                         |
| **Future Impact**                  | Potential for stronger early professional network; higher debt load; less flexibility if uncertain.    | Lower debt; more flexibility; may need to work harder to build network after transferring.      | Lower debt improves long-term financial stability. While networking starts later, Alex can still achieve her goals with intentional effort after transferring. The flexibility and reduced risk of community college outweigh the delayed networking.                                                                                       |

### **Recommendation**

**Based on the factors provided, Alex should start at a community college to earn an associate degree, then transfer to a four-year university to complete her bachelor’s degree.**

**Rationale:**  
- This path offers significant financial savings and minimizes student loan debt, which is important for long-term stability.
- Community college allows Alex to explore different majors at a lower cost, reducing the risk of expensive mistakes if she changes direction.
- While there is a delay in building a university network, transfer students can still access campus resources, internships, and professional connections after transferring.
- The flexibility and reduced financial risk of starting at community college outweigh the benefits of immediate university entry, especially given Alex’s current uncertainty about her major and career path.